In [2]:
import pandas as pd
import os
from PIL import Image
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten

In [4]:
import numpy as np

In [5]:
# Load the CSV file containing image labels
df = pd.read_csv('softec-aic-2024/Train.csv')

# Define directories for mitosis and normal images
mitosis_dir = 'softec-aic-2024/Training Images/Mitosis'
normal_dir = 'softec-aic-2024/Training Images/Normal'

# Define image size
img_size = (64, 64)

# Initialize empty lists for image data and labels
X = []
Y = []

# Load and preprocess images
for i in range(0, 226):
    filename = str(i) + '.jpg'

    # Check if image is in mitosis or normal folder
    if filename in os.listdir(mitosis_dir):
        label = 1
    else:
        label = 0

    # Load image and resize
    img = Image.open(os.path.join(mitosis_dir if label else normal_dir, filename))
    img = img.resize(img_size)

    # Convert image to numpy array and normalize
    img_array = np.array(img) / 255.0

    # Add image and label to lists
    X.append(img_array)
    Y.append(label)

In [6]:
from sklearn.model_selection import train_test_split

In [7]:
# Convert lists to numpy arrays
X = np.array(X)
Y = np.array(Y)

# Split data into training and testing sets
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Define the model
model = Sequential()
model.add(Conv2D(16, (3, 3), activation='relu', input_shape=(64, 64, 3)))
model.add(MaxPool2D())
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPool2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train the model
model.fit(X_train, Y_train, epochs=10, batch_size=32, validation_data=(X_test, Y_test))

# Evaluate the model
loss, accuracy = model.evaluate(X_test, Y_test)
print(f"Loss: {loss}, Accuracy: {accuracy}")




Epoch 1/10


6/6 [==============================] - 9s 429ms/step - loss: 0.7212 - accuracy: 0.6000 - val_loss: 0.6102 - val_accuracy: 0.7174
Epoch 2/10
6/6 [==============================] - 1s 203ms/step - loss: 0.5447 - accuracy: 0.8000 - val_loss: 0.5193 - val_accuracy: 0.7174
Epoch 3/10
6/6 [==============================] - 1s 255ms/step - loss: 0.4820 - accuracy: 0.7722 - val_loss: 0.4735 - val_accuracy: 0.8261
Epoch 4/10
6/6 [==============================] - 1s 149ms/step - loss: 0.4532 - accuracy: 0.8111 - val_loss: 0.4874 - val_accuracy: 0.8261
Epoch 5/10
6/6 [==============================] - 1s 210ms/step - loss: 0.4689 - accuracy: 0.8111 - val_loss: 0.4706 - val_accuracy: 0.8478
Epoch 6/10
6/6 [==============================] - 1s 169ms/step - loss: 0.4736 - accuracy: 0.7889 - val_loss: 0.4716 - val_accuracy: 0.8478
Epoch 7/10
6/6 [==============================] - 1s 196ms/step - loss: 0.4918 - accuracy: 0.7889 - val_loss: 0.5116 - val_accuracy: 0.7826
Epoch 8/10
6/6 

In [18]:
# Load the test images
test_images = []
for i in range(226,283):
    filename = str(i) + '.jpg'
    img = Image.open(os.path.join('softec-aic-2024/Testing Images', filename))
    img = img.resize(img_size)
    img_array = np.array(img) / 255.0
    test_images.append(img_array)

# Convert the test images to a NumPy array
test_images = np.array(test_images)

# Predict the labels for the test images
predictions = model.predict(test_images)


2/2 [==============================] - 0s 23ms/step


In [15]:
test = pd.read_csv('softec-aic-2024/Test.csv')
test['Label'] = predictions

# Convert probability predictions to class labels
test['Label'] = test['Label'].apply(lambda x: 'Mitosis' if x > 0.5 else 'Normal')


In [17]:
test.to_csv('predtion.csv')